## Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from tqdm.auto import tqdm

## Set Device

In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = "mps" if torch.backends.mps.is_available() else device

## Create fully connected layer

In [5]:
class GRU(nn.Module):
    def __init__(self, input_size=28, sequence_length=28, hidden_size=256, num_layers=2, num_classes=10):
        super(GRU, self).__init__()
        
        self.input_size = input_size
        self.sequence_length = sequence_length
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size*sequence_length, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out.contiguous().view(out.size(0), -1)
        out = self.fc(out)
        return out

In [6]:
model = GRU()
x = torch.randn(64, 28, 28)

print(model(x).shape)

torch.Size([64, 10])


## Hyperparameters

In [7]:
input_size = 28
sequence_length = 28
hidden_size = 256
num_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 20
grad_accum = 8

In [8]:
train_dataset = datasets.MNIST(root="data/", train=True, transform=transforms.ToTensor(), download=True)
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
train_dataset, train_dataloader

(Dataset MNIST
     Number of datapoints: 60000
     Root location: data/
     Split: Train
     StandardTransform
 Transform: ToTensor(),
 <torch.utils.data.dataloader.DataLoader at 0x10d2abbb0>)

In [9]:
test_dataset = datasets.MNIST(root="data/", train=False, transform=transforms.ToTensor(), download=True)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)
test_dataset, test_dataloader

(Dataset MNIST
     Number of datapoints: 10000
     Root location: data/
     Split: Test
     StandardTransform
 Transform: ToTensor(),
 <torch.utils.data.dataloader.DataLoader at 0x10d2abc40>)

## Initialize Network

In [11]:
model = GRU(input_size=input_size, sequence_length=sequence_length, hidden_size=hidden_size, num_layers=2, num_classes=num_classes)
model = model.to(device)

## Loss and Optimizer

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=lr)

## Train the Model

In [15]:
for epoch in range(num_epochs):
    accum = 0
    optimizer.zero_grad()
    model.train()
    for batch in tqdm(train_dataloader, total=len(train_dataloader), desc=f"Epoch {epoch}"):
        x, y = batch
        x = x.squeeze().to(device)
        y = y.to(device)
        y_hat = model(x)

        loss = criterion(y_hat, y) / grad_accum
        loss.backward()

        accum += 1
        if accum % grad_accum == 0:
            optimizer.step()
            optimizer.zero_grad()

    model.eval()
    test_losses = 0
    with torch.no_grad():
        for batch in tqdm(test_dataloader, total=len(test_dataloader), desc=f"Epoch {epoch}"):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            y_hat = model(x.squeeze())
    
            loss = criterion(y_hat, y)
            test_losses += loss

    print(f"Epoch {epoch}: Test loss {test_losses/len(test_dataloader):.2f}")

Epoch 0:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 0:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 0: Test loss 0.07


Epoch 1:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/157 [00:00<?, ?it/s]

...

Epoch 18:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 18: Test loss 0.04


Epoch 19:   0%|          | 0/938 [00:00<?, ?it/s]

Epoch 19:   0%|          | 0/157 [00:00<?, ?it/s]

Epoch 19: Test loss 0.04


In [18]:
def check_accuracy(loader, model):
    model.eval()
    cum_tp = 0
    cum_tot = 0
    with torch.no_grad():
        for batch in tqdm(loader, total=len(loader), desc=f"Epoch {epoch}"):
            x, y = batch
            x = x.to(device)
            y = y.to(device)
            y_hat = model(x.squeeze())

            preds = F.softmax(y_hat, dim=-1)
            _, pred_idx = preds.max(dim=-1)
            cum_tp += (pred_idx == y).sum()
            cum_tot += y.numel()

    print(f"Accuracy: {cum_tp/cum_tot*100:.4f}")

In [19]:
check_accuracy(train_dataloader, model)
check_accuracy(test_dataloader, model)

Epoch 19:   0%|          | 0/938 [00:00<?, ?it/s]

Accuracy: 99.8967


Epoch 19:   0%|          | 0/157 [00:00<?, ?it/s]

Accuracy: 99.2200
